In [2]:
using CSV
using DataFrames
using Dates
using Pipe
using Unitful

┌ Info: Precompiling Unitful [1986cc42-f94f-5a68-af5c-568840ba703d]
└ @ Base loading.jl:1278


In [11]:
DATA = "../../data"

"../../data"

In [13]:
rainfalllong = CSV.File("$DATA/rainfall_forecast.csv") |> DataFrame

,city,date,rainfall
,String,Date,Float64
1,Olecko,2020-11-16,2.9
2,Olecko,2020-11-17,4.1
3,Olecko,2020-11-19,4.3
4,Olecko,2020-11-20,2.0
5,Olecko,2020-11-21,0.6
6,Olecko,2020-11-22,1.0
7,Ełk,2020-11-16,3.9
8,Ełk,2020-11-19,1.2
9,Ełk,2020-11-20,2.0


In [14]:
transform!(rainfalllong, :rainfall => x -> x.* u"mm", renamecols=false)

,city,date,rainfall
,String,Date,Quantit…
1,Olecko,2020-11-16,2.9 mm
2,Olecko,2020-11-17,4.1 mm
3,Olecko,2020-11-19,4.3 mm
4,Olecko,2020-11-20,2.0 mm
5,Olecko,2020-11-21,0.6 mm
6,Olecko,2020-11-22,1.0 mm
7,Ełk,2020-11-16,3.9 mm
8,Ełk,2020-11-19,1.2 mm
9,Ełk,2020-11-20,2.0 mm


In [15]:
rainfallwide = unstack(rainfalllong, :date, :city, :rainfall)

,date,Olecko,Ełk
,Date,Quantit…?,Quantit…?
1,2020-11-16,2.9 mm,3.9 mm
2,2020-11-17,4.1 mm,missing
3,2020-11-19,4.3 mm,1.2 mm
4,2020-11-20,2.0 mm,2.0 mm
5,2020-11-21,0.6 mm,missing
6,2020-11-22,1.0 mm,2.0 mm


In [18]:
rainfallwide.Ełk

6-element Array{Union{Missing, Quantity{Float64,𝐋,Unitful.FreeUnits{(mm,),𝐋,nothing}}},1}:
 3.9 mm
       missing
 1.2 mm
 2.0 mm
       missing
 2.0 mm

In [19]:
alldays = DataFrame(date=Date.(2020, 11, 16:22))

,date
,Date
1,2020-11-16
2,2020-11-17
3,2020-11-18
4,2020-11-19
5,2020-11-20
6,2020-11-21
7,2020-11-22


In [23]:
@pipe leftjoin(alldays, rainfallwide, on=:date) |>
    coalesce.(_, 0.0u"mm")

,date,Olecko,Ełk
,Date,Quantit…,Quantit…
1,2020-11-16,2.9 mm,3.9 mm
2,2020-11-17,4.1 mm,0.0 mm
3,2020-11-18,0.0 mm,0.0 mm
4,2020-11-19,4.3 mm,1.2 mm
5,2020-11-20,2.0 mm,2.0 mm
6,2020-11-21,0.6 mm,0.0 mm
7,2020-11-22,1.0 mm,2.0 mm
